This program interfaces with Gab.com to scrap and analyze posts from a specific group.  I chose the group 'Science' because it is highly active with 90k members.  I trigger AJAX autoscroll several times to load more entries.

Initially I looked for uses of modal verbs such as 'could' and 'might' among the summary box for links posted in this group.  These are often used in sensational scientific/pseudoscientific article headlines to stir a response.  the rule of thumb I was taught is one should always immediately also reframe the headline with the 'not' version such as 'might not' to reframe the article and tamp down hype.  

It turns out there was not a huge amount of modal verb usage going on among link summaries.  I noticed many discussions still center around pandemic terms and found relatively more articles containing related terms.   

In [18]:
#Import:
# - Selenium webdriver to crawl/scrape the Phoenix Craigslist garage and moving sales section
# - Sleep to be respectful to the website provider and their customers
# - My own config file cdconfig to keep sensitive content out of GitHub
# - BeautifulSoup for parsing HTML content
# - Pandas for further data handlng and writing easily to a CSV
# - re for regular expression use during content evaluation

from selenium import webdriver
from time import sleep
import config_essentials
from bs4 import BeautifulSoup
import pandas as pd
import re

driver=webdriver.Chrome(executable_path=config_essentials.CHROMELOC)


C:\Users\elenc\AppData\Local\Temp/ipykernel_23888/895427273.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=config_essentials.CHROMELOC)


In [19]:
# head over to the Gab.com sign-in page
driver.get("https://gab.com/auth/sign_in")
sleep(2)

In [20]:
# log in
login_form = driver.find_element_by_id("user_email").send_keys(config_essentials.UNAME)
login_form = driver.find_element_by_id("user_password").send_keys(config_essentials.PWORD)
sleep(1)
try_click = driver.find_element_by_name("button").click()

C:\Users\elenc\AppData\Local\Temp/ipykernel_23888/205769183.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login_form = driver.find_element_by_id("user_email").send_keys(config_essentials.UNAME)
C:\Users\elenc\AppData\Local\Temp/ipykernel_23888/205769183.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login_form = driver.find_element_by_id("user_password").send_keys(config_essentials.PWORD)
C:\Users\elenc\AppData\Local\Temp/ipykernel_23888/205769183.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  try_click = driver.find_element_by_name("button").click()


In [21]:
# I am only in the 'science' group that has 90k members so head to my groups feed
driver.get("https://gab.com/groups")
sleep(2)

In [22]:
# scroll down 5x, triggering AJAX autoscroll each time to to load many more entries
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(5)

In [23]:
# the plan is to search for modal verbs such as: 'might', 'may', 'could'
# use of these terms is often a sensational practice since 'might not' and 'could not' are equally reasonable framings

modal_verbs  = ['possibly', 'can', 'could', 'may', 'might', 'should']

# I am specifically interested in articles linked to and will be mining the summary box for posted links
# the question - does gab/science discuss certain things or sensational-ish things?

bs = BeautifulSoup(driver.page_source, 'html.parser')

# grab the news feed column
block = bs.find(class_="_UuSG")

def check_link_preview_titles(search_terms, ret_dict={"TITLE":[], "TERMS":[]}):
    
    # if I do not clear the default dict it will accumulate values between calls
    ret_dict={"TITLE":[], "TERMS":[]}
    
    # use the bizarre Gab span class for link preview titles to gather them up
    for listing in block.find_all('span',{'class':'_UuSG _3rXHO _3_54N _1-fAn _2ZzNB _3hcKE _3aRhh'}):
        found = []
        ltext = listing.text.lower().strip()
        for term in search_terms:
            if re.search(r'\b{}\b'.format(term), ltext):
                found.append(term)
        if len(found)>0:
            ret_dict["TITLE"].append(ltext)
            ret_dict["TERMS"].append(" ".join(found))
    
    return ret_dict

# search for modal verbs ...
evidence = check_link_preview_titles(modal_verbs)
df       = pd.DataFrame.from_dict(evidence)
df.to_csv('essentials_modal_verbs.txt')
df

,TITLE,TERMS
0,rhythmic control of ’brain waves’ can boost me...,can
1,#enigma2022: contextual security should supple...,should
2,meet the college dropout and former bouncer wh...,might
3,climate engineering: weapon that can bring pop...,can
4,former blackrock executive says big pharma's l...,could
5,oura ring 3 review: can a smart ring replace y...,can


In [24]:
# that did not reveal a huge amount.  let's look for pandemic words
pandemic     = ['covid','coronavirus','pandemic','quarantine']
covid_terms  = check_link_preview_titles(pandemic)  
df2          = pd.DataFrame.from_dict(covid_terms)
df2.to_csv('essentials_pandemic.txt')

# this revealed a bit more.  when will this topic become less popular?
df2

,TITLE,TERMS
0,does vitamin d work on covid after all? israel...,covid
1,memphis: all citizens seeking emergency shelte...,covid
2,china’s locked-down city thrown into chaos aft...,covid
3,moderna’s covid vaccine for teens awaits ok as...,covid
4,rise in heart injuries for children below age ...,covid
5,dominican priest: covid vaccination is not a m...,covid
6,poll finds majority of canadians want covid re...,covid
7,nurse claims canadian pm justin trudeau’s covi...,covid
8,ann vandersteel slams hospitals profiting from...,covid pandemic
9,biden admin bribing doctors to push covid shot...,covid


In [25]:
# shut down our Chrome driver/instance
driver.quit()